In [22]:
import rdflib
from neo4j import GraphDatabase



In [23]:

from neo4j import GraphDatabase
import numpy as np


#uri = "bolt://localhost:7687"
#username = "neo4j"
#password = "neo4jkngbq"
uri="bolt://ec2-3-235-153-181.compute-1.amazonaws.com:7687"
username = "neo4j"
password = "somepass"
database = "neo4j"
driver = GraphDatabase.driver(uri, auth=(username, password))



def get_embeddings_nodes_by_type(tx, node_type):
    try: 
        cursor = tx.run("MATCH (n:" +node_type+") WHERE n.embeddings IS NOT NULL RETURN n")
        nodes_with_embeddings = []
        for record in cursor:
            node = record["n"]
            node_properties = node._properties
            
            #the embeddings is a string of floats with comma seperated, need to be converted to list of floats 
            embeddings = node_properties.get('embeddings')
            
            if("Interface" == node_type):
                nodes_with_embeddings.append({"node_id": node_properties.get('id'), "embeddings": embeddings})
            elif("SubNode" == node_type):
                nodes_with_embeddings.append({"node_id": node_properties.get('ID'), "embeddings": embeddings})
                
        return nodes_with_embeddings
    except Exception as e:
        raise Exception(f"Error retrieving nodes with embeddings: {e}")

def create_relation_between_nodes(tx, subNode_id, interface_id, relation="possible_threat"):


    query = f"""
            MATCH (from:SubNode {{ID:$subNode_id}}), (to:Interface {{id:$interface_id}})
            CREATE (from)-[r:{relation}] -> (to)
    """
    tx.run(query, subNode_id=subNode_id, interface_id=interface_id)
    print("Relation successfully created")
        
def convert_embedding_str_to_list(embedding_str):
    emb_split = embedding_str.split(',')
    emb_list = [float(x) for x in emb_split]
    return emb_list
        
def calculate_cosine_similarity(list1,list2):
    return np.dot(list1, list2) / (np.linalg.norm(list1) * np.linalg.norm(list2))



In [24]:
power_grid_nodes = []
mitre_grid_nodes = []

with driver.session() as session:
    power_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "Interface")
    mitre_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "SubNode")

/var/folders/kv/8ywy_8nd2z716q65sc27_81r0000gn/T/ipykernel_9045/4019469652.py:5: DeprecationWarning: write_transaction has been renamed to execute_write
  power_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "Interface")
/var/folders/kv/8ywy_8nd2z716q65sc27_81r0000gn/T/ipykernel_9045/4019469652.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  mitre_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "SubNode")


In [25]:
similarity_for_both_graphs = []

for power_node in power_grid_nodes:
    for attack_node in mitre_grid_nodes:
        node_similarity = {}
        node_similarity['power_node_id'] =power_node.get("node_id")
        node_similarity['attack_node_id'] =attack_node.get("node_id")        
        if(power_node.get("embeddings") and attack_node.get("embeddings")):
            power_node_emb = convert_embedding_str_to_list(power_node.get("embeddings"))
            attack_node_emb = convert_embedding_str_to_list(attack_node.get("embeddings"))
            cosine_similarity = calculate_cosine_similarity(power_node_emb, attack_node_emb)
            node_similarity['similarity'] = cosine_similarity
        similarity_for_both_graphs.append(node_similarity)


In [26]:
similarity_for_both_graphs.sort(key=lambda x: x['similarity'], reverse=True)

In [27]:
len(similarity_for_both_graphs)

110484

In [28]:
# create relation


for similarity_dict in similarity_for_both_graphs:
    if(similarity_dict.get('similarity') and similarity_dict.get('similarity')>0.75):
        attack_id = similarity_dict.get('attack_node_id')
        power_id = similarity_dict.get('power_node_id')
        if(None != attack_id and attack_id.startswith("S")): # added just to create relation b/w
            with driver.session() as session:
                session.write_transaction(create_relation_between_nodes,attack_id, power_id, "possible_attack")

/var/folders/kv/8ywy_8nd2z716q65sc27_81r0000gn/T/ipykernel_9045/1717615616.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relation_between_nodes,attack_id, power_id, "possible_attack")


Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation s

Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation s

Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation s

Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation s

Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation s